# Wolf, goat and cabbage problem

River crossing puzzle.

In [1]:
import numpy as np
import networkx as nx

from enum import Enum
from copy import deepcopy
from pprint import pprint

## Defining the game

In [2]:
class Location(Enum):
    A = 1
    B = 2

class State:
    man = Location.A
    wolf = Location.A
    goat = Location.A
    cabbage = Location.A

    def __init__(self, man, wolf, goat, cabbage):
        self.man = man
        self.wolf = wolf
        self.goat = goat
        self.cabbage = cabbage

    def __eq__(self, other):
        if (
            self.man == other.man
            and self.wolf == other.wolf
            and self.goat == other.goat
            and self.cabbage == other.cabbage
        ):
            return True
        return False

    def __hash__(self):
        return hash((self.man, self.wolf, self.goat, self.cabbage))

    def __repr__(self):
        return f"{self.man},{self.wolf},{self.goat},{self.cabbage}"


Let's first define the start and end state

In [3]:
initial_state = State(Location.A, Location.A, Location.A, Location.A)
goal_state = State(Location.B, Location.B, Location.B, Location.B)

In [4]:
class Graph:
    def __init__(self):
        self.edges = {}

    def neighbors(self, node):
        return self.edges[node]


graph = Graph()

In [8]:
def is_valid(state):
    if state.goat == state.cabbage and state.man != state.goat:
        return False
    if state.wolf == state.goat and state.man != state.wolf:
        return False
    return True


def next_states(state):
    current = state.man
    next = Location.B if state.man == Location.A else Location.A
    moves = []
    # Include the farmer moving by himself
    alone = deepcopy(state)
    alone.man = next
    moves.append(alone)

    for thing in ["wolf", "goat", "cabbage"]:
        if getattr(state, thing) == state.man:
            copy = deepcopy(state)
            setattr(copy, thing, next)
            copy.man = next
            moves.append(copy)

    return moves

In [9]:
# Encode all states into the graph
def encode(state):
    if state == goal_state:
        return
    if state not in graph.edges.keys():
        nodes = next_states(state)
        graph.edges[state] = nodes
        for node in nodes:
            encode(node)

In [10]:
encode(initial_state)

In [11]:
# Check that graph is encoded
pprint(graph.edges)

{Location.B,Location.B,Location.A,Location.A: [Location.A,Location.B,Location.A,Location.A,
                                               Location.A,Location.A,Location.A,Location.A],
 Location.B,Location.A,Location.A,Location.A: [Location.A,Location.A,Location.A,Location.A],
 Location.A,Location.A,Location.A,Location.A: [Location.B,Location.A,Location.A,Location.A,
                                               Location.B,Location.B,Location.A,Location.A,
                                               Location.B,Location.A,Location.B,Location.A,
                                               Location.B,Location.A,Location.A,Location.B],
 Location.A,Location.B,Location.B,Location.A: [Location.B,Location.B,Location.B,Location.A,
                                               Location.B,Location.B,Location.B,Location.B],
 Location.B,Location.B,Location.B,Location.A: [Location.A,Location.B,Location.B,Location.A,
                                               Location.A,Location.A,Locatio

# BFS

In [32]:
def bfs(root):
    queue = [[root]]
    visited = set([])
    i=0
    while len(queue) != 0:
        path = queue.pop()
        node = path[-1]
        if node == goal_state:
            return path
        for edge in graph.edges[node]:
            if edge not in visited:
                visited.add(edge)
                new_path = list(path)
                new_path.append(edge)
                queue.append(new_path) 
        i+=1
        print("interations:", i)

In [33]:
bfs(initial_state)

interations: 1
interations: 2
interations: 3
interations: 4
interations: 5
interations: 6
interations: 7
interations: 8
interations: 9


[Location.A,Location.A,Location.A,Location.A,
 Location.B,Location.A,Location.A,Location.B,
 Location.A,Location.A,Location.A,Location.B,
 Location.B,Location.A,Location.B,Location.B,
 Location.A,Location.A,Location.B,Location.A,
 Location.B,Location.B,Location.B,Location.A,
 Location.A,Location.B,Location.B,Location.A,
 Location.B,Location.B,Location.B,Location.B]

# DFS

In [12]:
def dfs(root):
    stack = [[root]]
    visited = set([])
    i=0
    while len(stack) != 0:
        path = stack.pop(len(stack)-1)
        node = path[-1]
        visited.add(node)
        if node == goal_state:
            return path
        for edge in graph.edges[node]:
            if edge not in visited:
                visited.add(edge)
                new_path = list(path)
                new_path.append(edge)
                stack.append(new_path)
        i+=1
        print("interations: i

In [13]:
dfs(initial_state)

[Location.A,Location.A,Location.A,Location.A,
 Location.B,Location.A,Location.B,Location.A,
 Location.A,Location.A,Location.B,Location.A,
 Location.B,Location.A,Location.B,Location.B,
 Location.A,Location.A,Location.A,Location.B,
 Location.B,Location.B,Location.A,Location.B,
 Location.A,Location.B,Location.A,Location.B,
 Location.B,Location.B,Location.B,Location.B]